In [11]:
import numpy as np
import matplotlib
from itertools import groupby
from operator import itemgetter
from matplotlib import pyplot as plt
from hdf5manager import hdf5manager as h5
from scipy.stats import mode
#from derivativeEventDetection import detectSpikes, FixedQueue
#from eventCoincidence import eventCoin, _displayInfo, visualizeProgress
import ctypes as c
from multiprocessing import Process, Array, cpu_count, Manager
import pandas as pd
import time
from waveletAnalysis import waveletAnalysis as wave
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [12]:
def batchBinarize(  ROI_timecourses, #
                    fps = 10,
                    window_size = 50, 
                    plot = False
                    ):
    
    print('Binarizing ROI_timecourses.')
    
    binarizedData = np.zeros_like(ROI_timecourses, dtype='uint8')
    eventMat = np.zeros_like(ROI_timecourses, dtype='uint8')

    for i , datarow in enumerate(ROI_timecourses):
        # start_time = time.clock()
        w = wave(datarow, fps=fps, siglvl = 0.99)
        spikes, vals = w.waveletEventDetection()
        e = []
        spike_del = []
        for j, v in enumerate(vals):
            found = False
            possible = np.where(np.array(0.25*v < ROI_timecourses[i,spikes[j]-window_size :spikes[j]+window_size ]))[0] + spikes[j] - window_size 
            for k, g in groupby(enumerate(possible), lambda x: x[0] - x[1]):
                l = list(map(itemgetter(1), g))
                if spikes[j] in l:
                    found = True
                    e.extend(l)
            if not found:
                spike_del.append(j)
        spikes = np.delete(spikes, spike_del)
        binarizedData[i,spikes] = 1
        eventMat[i,np.unique(e)] = 1

    if plot:
        t = np.arange(ROI_timecourses.shape[1])/fps

        fig = plt.figure(figsize=(10,6))
        gs = GridSpec(5, 4) # 2 rows, 3 columns
        ax1=fig.add_subplot(gs[0,:]) 
        ax2=fig.add_subplot(gs[1,:], sharex=ax1)
        ax3=fig.add_subplot(gs[2,:], sharex=ax1)
        ax4=fig.add_subplot(gs[3,:], sharex=ax1)
        ax5=fig.add_subplot(gs[4,:], sharex=ax1)

        ax1.plot(t, ROI_timecourses[0] * 100, label='dFoF', c = 'b')
        ax1.plot(t, eventMat[0]+1, label = 'eventMat', c = 'r')
        ax1.eventplot(np.where(binarizedData[0] > 0.5)[0]* 1/fps, label = 'waveEvent', lineoffsets = -1, linelengths=0.5, color='k')
        ax1.set_ylabel('dfof {0}'.format(0))
        plt.setp(ax1.get_xticklabels(), visible=False)

        # ax2.plot(t, ROI_timecourses[1], label='dFoF', c = 'b')
        # ax2.plot(t, eventMat[1]+1, label = 'eventMat', c = 'r')
        # ax2.eventplot(np.where(binarizedData[1] > 0.5)[0] * 1/fps, label = 'waveEvent', lineoffsets = -1, linelengths=0.5, color='k')
        # ax2.set_ylabel('dfof {0}'.format(1))
        # plt.setp(ax2.get_xticklabels(), visible=False)

        # ax3.plot(t, ROI_timecourses[2], label='dFoF', c = 'b')
        # ax3.plot(t, eventMat[2]+1, label = 'eventMat', c = 'r')
        # ax3.eventplot(np.where(binarizedData[2] > 0.5)[0] * 1/fps, label = 'waveEvent', lineoffsets = -1, linelengths=0.5, color='k')
        # ax3.set_ylabel('dfof {0}'.format(2))
        # plt.setp(ax3.get_xticklabels(), visible=False)

        # ax4.plot(t, ROI_timecourses[3], label='dFoF', c = 'b')
        # ax4.plot(t, eventMat[3]+1, label = 'eventMat', c = 'r')
        # ax4.eventplot(np.where(binarizedData[3] > 0.5)[0] * 1/fps, label = 'waveEvent', lineoffsets = -1, linelengths=0.5, color='k')
        # ax4.set_ylabel('dfof {0}'.format(3))
        # plt.setp(ax4.get_xticklabels(), visible=False)

        # ax5.plot(t, ROI_timecourses[4], label='dFoF', c = 'b')
        # ax5.plot(t, eventMat[4]+1, label = 'eventMat', c = 'r')
        # ax5.eventplot(np.where(binarizedData[4] > 0.5)[0] * 1/fps, label = 'waveEvent', lineoffsets = -1, linelengths=0.5, color='k')
        # ax5.set_ylabel('dfof {0}'.format(4))
        # ax5.set_xlabel('time (s)')

        plt.tight_layout()
        plt.show()

    return binarizedData, eventMat

In [14]:
#Characterizes each timecourse in a matrix of brain data by number of events, event frequency, maximum event magnitude
#and average inter-event interval (time between consecutive events). Each event in each timecourse is further characterized
#by its magnitude and duration
def eventCharacterization(ROI_timecourses, binarizedData, eventMat):
    # max_events = 0
    # min_events = 1000

    print('Characterizing events.')

    def findMiddle(input_list):
        middle = float(len(input_list))/2
        if middle % 2 != 0:
            return input_list[int(middle - .5)]
        else:
            return input_list[int(middle)]

    combineEvent = eventMat + binarizedData

    m_dict = dict(event_dur = [0],
                interevent_length = [0])
    master = [dict() for x in range(ROI_timecourses.shape[0])]
    for i, eventRow in enumerate(combineEvent):
        eventFrames = np.where(eventRow > 0.5)[0]
        j=0
        for k, g in groupby(enumerate(eventFrames), lambda x: x[0] - x[1]):
            l = list(map(itemgetter(1), g))
            if j == 0:
                master[i]['start_event_loc'] = [l[0]]
                master[i]['mid_event_loc'] = [findMiddle(l)]
                master[i]['inter_event_fr_dur'] = [l[0]]
                master[i]['event_fr_dur'] = [len(l)]
                master[i]['n_wave_events'] = [np.sum([eventRow[l] > 1])]
                master[i]['event_max_amp'] = [np.max(ROI_timecourses[i,l])]
                master[i]['event_min_amp'] = [np.min(ROT_timecourses[i,1])]
                master[i]['event_avg_amp'] = [np.mean(ROI_timecourses[i,l])]
                ''' 
                should we cluster these after characterizing to determine burst and single events.  These parameters will change
                with respect to age.
                '''
                # if master[i]['n_wave_events'][-1] < 3: 
                if master[i]['n_wave_events'][-1] < 3 or (master[i]['event_fr_dur'][-1] < 20 and master[i]['n_wave_events'][-1] == 2):
                    master[i]['event_type'] = ['s'] # single
                else:
                    master[i]['event_type'] = ['b'] # burst
                prev_last_ind = l[-1]
            else:
                master[i]['start_event_loc'].append(l[0])
                master[i]['mid_event_loc'].append(findMiddle(l))
                master[i]['inter_event_fr_dur'] = [l[0]-1-prev_last_ind]
                master[i]['event_fr_dur'].append(len(l))
                master[i]['n_wave_events'].append(np.sum([eventRow[l] > 1]))
                master[i]['event_max_amp'].append(np.max(ROI_timecourses[i,l]))
                master[i]['event_min_amp'].append(np.min(ROT_timecourses[i,1]))
                master[i]['event_avg_amp'].append(np.mean(ROI_timecourses[i,l]))
                # if master[i]['n_wave_events'][-1] < 3:
                if master[i]['n_wave_events'][-1] < 3 or (master[i]['event_fr_dur'][-1] < 20 and master[i]['n_wave_events'][-1] == 2):
                    master[i]['event_type'].append('s') # single
                else:
                    master[i]['event_type'].append('b') # burst
                prev_last_ind = l[-1]
            j+=1 

        master[i]['n_events'] = j
    # for row in master:               
    #     print(row)
    # print(master[0].keys())
    return master

In [1]:
path2 = '/Users/shreyamantripragada/Documents/brain_data.hdf5'
k = h5(path2)
k.keys()
dfof = k.load('dfof_mean')


binarizedData, eventMat = batchBinarize(dfof)
# plt.plot(eventMat[0])

win_size = 100

win_mean = np.convolve(dfof, np.ones(win_size)/win_size, mode = 'same')

flat_dfof = dfof - win_mean

threshold = np.zeros(dfof.shape)
threshold[dfof>np.std(dfof)] = 1

plt.plot(dfof)
plt.plot(win_mean)
plt.plot(flat_dfof)
plt.show()


NameError: name 'h5' is not defined

In [5]:
#!/usr/bin/env python3
'''
Functions for classifying ICA domains

Authors: Sydney C. Weiser and Brian R. Mullen
Date: 2019-04-06
'''

import wholeBrain as wb
import numpy as np
import scipy
# import math
# import seaborn as sns
#from sklearn.externals import joblib

from skimage.measure import label, regionprops
from waveletAnalysis import waveletAnalysis as wave

from matplotlib import pyplot as plt


try:
    from hdf5manager import hdf5manager as h5
except Exception as e:
    print('Error importing hdf5manager.py')
    print('\t ERROR : ', e)

try:
    import timecourseAnalysis as tca
except Exception as e:
    print('Error importing timecourseAnalysis.py')
    print('\t ERROR : ', e)


def sortNoise(timecourses, return_logpdf=False, method='KDE'):
    '''
    Sorts timecourses into two clusters (signal and noise) based on 
    lag-1 autocorrelation.  
    Timecourses should be a np array of shape (n, t).

    Returns noise_components, a np array with 1 value for all noise 
    timecourses detected, as well as the cutoff value detected
    '''
    noise_components = np.zeros((timecourses.shape[0],), dtype='uint8')

    if method == 'KDE':
        from sklearn.neighbors import KernelDensity
        from scipy.signal import argrelextrema

        # calculate lag autocorrelations
        lag1 = tca.lagNAutoCorr(timecourses, 1)

        # calculate minimum between min and max peaks
        kde_skl = KernelDensity(
            kernel='gaussian', bandwidth=0.05).fit(lag1[:, np.newaxis])
        x_grid = np.linspace(-0.2,1.2,1200)

        log_pdf = kde_skl.score_samples(x_grid[:, np.newaxis])

        maxima = argrelextrema(np.exp(log_pdf), np.greater)[0]
        if len(maxima) <= 1:
            print('Only one cluster found')
            cutoff = 0
        else:
            cutoff_index = np.argmin(np.exp(log_pdf)[maxima[0]:maxima[-1]]) \
                + maxima[0]
            cutoff = x_grid[cutoff_index]
            print('autocorr cutoff:', cutoff)

        noise_components[:] = (lag1 < cutoff).astype('uint8')
    else:
        raise Exception('method: {0} is unknown!'.format(method))

    if return_logpdf:
        return noise_components, cutoff, log_pdf
    else:
        return noise_components, cutoff


def findContinBool(boolArray1D):
    nestDict = {}
    j, k = (-1, 0)
    usek = True
    for i in range(len(boolArray1D)):
        if boolArray1D[i] and not boolArray1D[i-1]:
            j += 1 
            if j >= 1:
                nestDict['region' + str(j-1)]['length'] = k
            k = 0
            nestDict['region' + str(j)] = {}
            nestDict['region' + str(j)]['freq.index'] = [] 
            nestDict['region' + str(j)]['freq.index'].append(i)
            k += 1
        if boolArray1D[i] and boolArray1D[i-1]:
            if i == 1:
                j += 1
                nestDict['region' + str(j)] = {}
                nestDict['region' + str(j)]['freq.index'] = []
                nestDict['region' + str(j)]['freq.index'].append(i-1)
                nestDict['region' + str(j)]['freq.index'].append(i)
            elif i == 0:
                pass
            else:
                k += 1
                nestDict['region' + str(j)]['freq.index'].append(i)

    if j != -1:
        nestDict['region' + str(j)]['length'] = k
    
    return nestDict


def findSig():
    ratio = np.squeeze(w.gws/w.gws_sig)
    index = (ratio > 1)
    return ratio, index


def approxIntegrate(index, freq, power, sigcutoff): 
    #power
    diff = np.squeeze(power - sigcutoff) #integrate only significant area
    #freq
    if index[-1] != freq.shape[0]-1: # default to right sided estimation
        offset = [x+1 for x in index]
        binsz = freq[index] - freq[offset]
    else:
        offset = [x-1 for x in index]
        binsz = freq[offset] - freq[index]
        
    return np.sum(binsz * diff[index])


def temporalCharacterize(sigfreq, ratio, freq):
    for k in sigfreq.keys():
        sigfreq[k]['freq.maxsnr'] = np.nanmax(ratio[sigfreq[k]['freq.index']])
        sigfreq[k]['freq.maxsnr.freq'] = np.nanargmax(ratio[sigfreq[k]['freq.index']])
        sigfreq[k]['freq.avgsnr'] = np.nanmean(ratio[sigfreq[k]['freq.index']])
        sigfreq[k]['freq.range.low'] = freq[sigfreq[k]['freq.index'][-1]] 
        sigfreq[k]['freq.range.high'] = freq[sigfreq[k]['freq.index'][0]]
        if sigfreq[k]['freq.range.low'] and not sigfreq[k]['freq.range.high']:
                sigfreq[k]['freq.range.high'] = 5
        if sigfreq[k]['freq.range.high'] and not sigfreq[k]['freq.range.low']:
                sigfreq[k]['freq.range.low'] = 0                
        sigfreq[k]['freq.rangesz'] = (sigfreq[k]['freq.range.high'] - sigfreq[k]['freq.range.low'])
        sigfreq[k]['freq.integrate'] = approxIntegrate(index = sigfreq[k]['freq.index'], freq = w.flambda, 
                                                  power = w.gws, sigcutoff = w.gws_sig)

    return sigfreq


def sortNestedDict(nestDict, sortkey = 'freq.rangesz'):
    sortDict = []
    for k in nestDict.keys(): sortDict.append(k)
#     sortDict = sorted(sortDict, key=lambda x: (sigfreq[x]['length']), reverse =True)
    sortDict = sorted(sortDict, key=lambda x: (nestDict[x][sortkey]), reverse =True)

    return sortDict


def centerOfMass(eigenbrain, threshold=None, verbose=False, plot=False):
    eigen = eigenbrain.copy()
    if threshold is not None:
        eigen[eigen < threshold] = np.nan
    x, y = np.where(np.isnan(eigen)==False)

    #total sum
    totalmass = np.nansum(np.abs(eigen))
    #weighted sum
    sumrmx = 0
    sumrmy = 0
    for i in range(x.shape[0]):
        sumrmx += np.abs(eigenbrain[x[i],y[i]])*x[i]
        sumrmy += np.abs(eigenbrain[x[i],y[i]])*y[i]

#     plt.imshow(eigen)
#     plt.show()
    
    comx = sumrmx/totalmass
    comy = sumrmy/totalmass
   
    if verbose:
        print('xcom: ', comx)
        print('ycom: ', comy)

    if plot:
        plt.imshow(eigenbrain)
        plt.colorbar()
        plt.scatter(comy, comx, color='w', marker='*' )
        plt.show()

    return comx, comy


def xyProjectMax(eigenbrain, verbose=True, plot=True):
    xmean = np.nanmean(eigenbrain, axis = 1)
    xmax = np.nanargmax(xmean)
    
    ymean = np.nanmean(eigenbrain, axis = 0)
    ymax = np.nanargmax(ymean)

    if verbose:
        print('xmax: ', xmax)
        print('ymax: ', ymax)
        
    if plot:
        plt.plot(ymean)
        plt.plot(xmean)
        plt.scatter([xmax,ymax],[0,0], color = 'r')
        plt.show()
        
        
        plt.imshow(eigenbrain)
        plt.colorbar()
        plt.scatter(ymax, xmax, color='w', marker='*' )
        plt.show()

    return xmax, ymax

    
def positionMaxIntensity(eigenbrain, verbose=True, plot=True):
    amax = np.nanmax(eigenbrain)
    xamax, yamax = np.where(eigenbrain == amax)
    if verbose:
        print('x amax: ', xamax)
        print('y amax: ', yamax)
        
    if plot:
        plt.imshow(eigenbrain)
        plt.colorbar()
        plt.scatter(yamax, xamax, color='w', marker='*' )
        plt.show()
    return xamax, yamax
    

def spatialCharacterize(eigenbrain, pc, threshold, verbose = False, plot = False):
    eigen = eigenbrain.copy()
#     eigen[eigen == np.nan] = 0
    eigen[eigen < threshold] = np.nan  
    x, y = np.where(np.isnan(eigen)==False)
    image = np.zeros_like(eigenbrain)
    image[x,y] = 1
    image = scipy.ndimage.median_filter(image, size=5)
    label_img = label(image)
    regions = regionprops(label_img)
    totalmass = np.nansum(np.abs(eigenbrain))
    domregion = {}
    
    for i, props in enumerate(regions):
        domregion['region' + str(i)] = {}
        regcoord = props.coords
        intensity = np.zeros_like(regcoord[:,0]).astype('float16')
        for j, coord in enumerate(regcoord):
            intensity[j] = eigenbrain[coord[0], coord[1]]
        
        if plot:
            plt.scatter(y, x, color = (0,0,0,0.05), marker='.')
            plt.scatter(regcoord[:,1], regcoord[:,0], c=intensity, cmap='jet')
            plt.gca().invert_yaxis()
            plt.show()
        
#         domregion['region' + str(i)]['pc_id']=pc
        domregion['region' + str(i)]['threshold.area']=props.area
        domregion['region' + str(i)]['threshold.perc']=props.area/np.sum(image)
        domregion['region' + str(i)]['mass.total']=totalmass
        domregion['region' + str(i)]['mass.region']=np.nansum(intensity)
        domregion['region' + str(i)]['mass.perc']=np.nansum(intensity)/totalmass
        domregion['region' + str(i)]['region.centroid.0']=props.centroid[0]
        domregion['region' + str(i)]['region.centroid.1']=props.centroid[1]
        domregion['region' + str(i)]['region.orient']=props.orientation
        domregion['region' + str(i)]['region.majaxis']=props.major_axis_length
        domregion['region' + str(i)]['region.minaxis']=props.minor_axis_length
        domregion['region' + str(i)]['region.extent']=props.extent
        domregion['region' + str(i)]['region.eccentricity']=props.eccentricity
        if props.minor_axis_length > 0:
            domregion['region' + str(i)]['region.majmin.ratio']=props.major_axis_length/props.minor_axis_length

        if verbose:
            print('Threshold area: ', props.area)
            print('Percent threshold area assessed: ', props.area/np.sum(image))
            print('\nTotalmass: ', totalmass)
            print('Areamass: ', np.nansum(intensity))
            print('Percent mass: ', np.nansum(intensity)/totalmass)
            print('\nCentroid: ', props.centroid)
            print('Orientation :', props.orientation)
            print('Major axis: ', props.major_axis_length)
            print('Minor axis: ', props.minor_axis_length)
            print('Extent: ',props.extent)
            print('Eccentricty: ',props.eccentricity)

    return domregion


if __name__ == '__main__':

    import argparse
    import time
    import pandas 
    from wholeBrainPCA import rebuildEigenbrain

    # Argument Parsing
    # -----------------------------------------------
    ap = argparse.ArgumentParser()
    ap.add_argument('-i', '--input', type = argparse.FileType('r'), 
        nargs = '+', required = False, 
        help = 'path to the processed ica file(s)')
    ap.add_argument('-f', '--fps', default = 10, required = False,
        help = 'frames per second from recordings')
    ap.add_argument('-n', '--noise_only', action='store_true',
        help = 'deterime noise components and save in hdf5')
    ap.add_argument('-t','--train', action='store_true',
        help = 'train the classifier on the newest class_metric dataframe')
    ap.add_argument('-ud', '--updateDF', action='store_true',
        help = 'update full classifier dataframe')
    ap.add_argument('-uc', '--updateClass', action='store_true',
        help = 'update class in experimental dataframe, input ica.hdf5 and ensure metrics.tsv is in the same folder')
    ap.add_argument('-fc', '--force', action='store_true',
        help = 'force re-calculation')
    ap.add_argument('-p', '--plot', action='store_true',
        help= 'Vizualize training outcome')
    args = vars(ap.parse_args())

    classMetricsPath = '/home/brian/Documents/data/Classifier/class_metrics.tsv'
    # classifier = '/home/brian/Documents/data/Classifier/p21_classifier.sav'
    classifier = '/home/brian/Documents/data/Classifier/p21_classifier.hdf5'

    if args['input'] != None:
        paths = [path.name for path in args['input']]
        print('Input found:')
        [print('\t'+path) for path in paths]

        for path in paths:
            print('Processing file:', path)

            if path.endswith('.hdf5'):
                assert path.endswith('_ica.hdf5') | path.endswith('_ica_reduced.hdf5'),\
                     "Path did not end in '_ica.hdf5'"
                savepath = path.replace('.hdf5', '_metrics.tsv')
                savepath = savepath.replace('_reduced', '')
                base = os.path.basename(path)

                print('\nLoading data to create classifier metrics\n------------------------------------------------')
                f = h5(path)
                f.print()

                if ('noise_components' not in f.keys()) | args['force']:
                    print('Calculating Noise Components')
                    noise, cutoff = sortNoise(f.load('timecourses'))
                    f.save({'noise_components':noise, 'cutoff':cutoff})
                else:
                    #Load data from ica.hdf5 file 
                    noise = f.load('noise_components')

                if args['noise_only']:
                    continue

                if 'artifact_components' in f.keys():
                    artifact = f.load('artifact_components')
                    if np.sum(artifact) != 0:
                        comb = noise + artifact
                        artifact[comb == 2] = 0 #if it is id'd as noise and artifact, keep as noise 
                        signal = np.array(comb == 0).astype(int)
                    else:
                        signal = np.zeros_like(noise)
                else:
                    artifact = np.zeros_like(noise)
                    signal = np.zeros_like(noise)

                notnoise_index = (noise==0)

                if args['updateClass'] and np.sum(signal)!=0 and os.path.exists(savepath):
                    data = pd.DataFrame.from_csv(savepath, sep = '\t', index_col='exp_ic')
                    print (data)
                    print('\nImporting dataframe\n------------------------------------')
                    print('Sum of each component BEFORE update:\n',  data[['artifact','signal']].sum())
                    data['artifact'] = artifact[notnoise_index]
                    data['signal'] = signal[notnoise_index]
                    print('\nSum of each component AFTER update:\n', data[['artifact','signal']].sum())
                                    #Save file for future manipulations
                    print('Saving dataframe to:', savepath)
                    data.to_csv(savepath, sep = '\t')

                if (os.path.exists(savepath) and args['force']) or (not os.path.exists(savepath)):
                    if args['force']:
                        print('Re-calculating the metrics.')
                    if args['updateClass']:
                        print('Unable to update experimental dataframe. Either no artifact components or no metrics.tsv found')
                        print('Continuing on to create dataframe')
                    flipped = f.load('flipped')
                    tcourse = f.load('timecourses')
                    roimask = f.load('roimask')
                    eig_vec = f.load('eig_vec')
                    thresh = f.load('thresholds')
                    try:
                         meta = f.load('expmeta')
                    except Exception as e:
                        print('Unable to add age to the dataFrame')
                        print('\t ERROR : ', e)                   

                    #flipped the inverted timeseries and 
                    tcourse = (np.multiply(tcourse.T, flipped)).T
                    eig_vec = np.multiply(eig_vec, flipped)

                    #create the dataframe and set up indices
                    data = pd.DataFrame()


                    data['artifact'] = artifact[notnoise_index]
                    data['signal'] = signal[notnoise_index]
                    data['age'] = np.ones(int(signal[notnoise_index].shape[0])) * int(re.findall(r'\d+',meta['meta']['anml']['age'])[0])
                    data['exp_ic'] = [base[:-9] + '-' + '{}'.format(str(i).zfill(4)) for i in np.where(noise == 0)[0].tolist()]
                    data = data.set_index('exp_ic')

                    print('\nCalculating spatial metrics\n------------------------------------------------')

                    data['spatial.std'] = np.nanstd(eig_vec[:, notnoise_index], axis = 0)
                    data['spatial.avg'] = np.nanmean(eig_vec[:, notnoise_index], axis = 0)
                    data['spatial.min'] = np.nanmin(eig_vec[:, notnoise_index], axis = 0)
                    data['spatial.max'] = np.nanmax(eig_vec[:, notnoise_index], axis = 0)

                    for i, pid in enumerate(data.index.tolist()):
                        if i%50 == 0:
                            print('Working on {0} of {1} components'.format(i, len (data)))
                        pc_id = i
                        
                        #rebuild the eigenbrain
                        eigenbrain = rebuildEigenbrain(eig_vec, pc_id, roimask=roimask, eigb_shape=None, 
                            maskind=None)
                        
                        comxall, comyall = centerOfMass(eigenbrain)
                        comxdom, comydom = centerOfMass(eigenbrain, threshold=thresh[i])
                        
                        #characterize the largest domain in the threshold
                        k = spatialCharacterize(eigenbrain, pc=pc_id, threshold=thresh[i], plot = False)

                        l = sortNestedDict(k, sortkey = 'mass.perc')
                        data.at[pid,'spatial.n.domains'] = len(l)
                        if len(l) > 0:
                            for key, val in k[l[0]].items():
                                data.at[pid, key] = val
              
                        data.at[pid,'spatial.COMall.x'] = comxall      
                        data.at[pid,'spatial.COMall.y'] = comyall        
                        data.at[pid,'spatial.COMdom.x'] = comxdom      
                        data.at[pid,'spatial.COMdom.y'] = comydom

                    print('\nCalculating temporal metrics\n------------------------------------------------')

                    data['temporal.autocorr'] = tca.lagNAutoCorr(tcourse[notnoise_index,:], 1)
                    data['temporal.min'] = np.nanmin(tcourse[notnoise_index,:], axis = 1)
                    data['temporal.max'] = np.nanmax(tcourse[notnoise_index,:], axis = 1)
                    data['temporal.std'] = np.nanstd(tcourse[notnoise_index,:], axis = 1)

                    for i, pid in enumerate(data.index.tolist()):
                        if i%50 == 0:
                            print('Working on {0} of {1} components'.format(i, len(data)))
                        w = wave(data = tcourse[i], fps = args['fps'], mother = 'MORLET',
                                 param = 4, siglvl = 0.99, verbose = False, plot=False)
                        w.globalWaveletSpectrum()
                        
                        ratio, index = findSig()
                        m = findContinBool(index)
                        m = temporalCharacterize(m, ratio, w.flambda)
                        n = sortNestedDict(m, sortkey = 'freq.rangesz')
                        
                        data.at[pid,'temporal.n.freq'] = len(n)
                        if len(n) > 0:
                            for key, val in m[n[0]].items():
                                if key == 'freq.index':
                                    pass
                                else:
                                    data.at[pid, key] = val

                    #Save file for future manipulations
                    print('Saving dataframe to:', savepath)
                    data.to_csv(savepath, sep = '\t')

            elif path.endswith('.tsv'):
                try:
                    data = pd.DataFrame.from_csv(path, sep = '\t', index_col='exp_ic')
                    print('Importing dataframe\n------------------------------------')
                    print(data.head())
                    signal = data['signal']
                except Exception as e:
                    print('Error importing dataFrame')
                    print('\t ERROR : ', e)
            
            elif path.endswith('.csv'):
                try:
                    data = pd.DataFrame.from_csv(path, sep = ',', index_col='exp_ic')
                    print('Importing dataframe\n------------------------------------')
                    print(data.head())
                    signal = data['signal']
                except Exception as e:
                    print('Error importing dataFrame')
                    print('\t ERROR : ', e)
            
            else:
                base = os.path.basename(path)
                print('\nFile type {} not understood.'.format(base))
                print('skipping ' + path + '\n')

            if np.sum(signal)==0:
                if path.endswith('.tsv'):
                    hdf5path = path.replace('_metrics.tsv', '.hdf5')
                    savepath = path

                elif path.endswith('.hdf5'):
                    hdf5path = path
                    savepath = path.replace('.hdf5', '_metrics.tsv')

                #put the dataframe on a scale 0 to 1 (expect age) and select only non-noise
                datacopy = data.drop('age', axis =1).copy()
                datacopy -= datacopy.min()
                datacopy /= datacopy.max()
                datacopy['age'] = data['age']

                X = datacopy.fillna(value=0).loc[:, new_vars]

                #load classfier hdf5
        
                g = h5(classifier)
                voting_clf = g.load('voting_clf')
                new_vars = g.load('training_keys')      

                # # load through joblib
                # voting_clf = joblib.load(classifier)
                # new_vars = [#spetial metrics 
                        # 'spatial.std', 'spatial.min', 'spatial.max', 'mass.perc', 
                        # 'region.minaxis', 'region.majaxis', 'region.orient', 
                        # 'threshold.area', 'region.eccentricity',
                        # #temporal metrics
                        # 'temporal.max', 'freq.maxsnr.freq', 'length', 'freq.rangesz']

                #run classifier
                data.loc[:, 'signal'] = voting_clf.predict(X)

                data['artifact'] = np.array(data['signal'] == 0).astype(int)
                data.sort_index()

                f = h5(hdf5path)
                noise = f.load('noise_components')
                print(noise)
                artifact = noise.copy() * 0
                artifact[noise==0] = data['artifact'].values.tolist()
                print(artifact)
                f.save({'artifact_components':artifact})

                print(len(artifact))
                print(data.head())

                #Save file for future manipulations
                print('Saving dataframe to:', savepath)
                data.to_csv(savepath, sep = '\t')

            if args['updateDF']:
                #open dataframe
                try:
                    main_data = pd.DataFrame.from_csv(classMetricsPath, sep = '\t', index_col='exp_ic')
                    print('Importing dataframe\n------------------------------------')
                    print(main_data.head())
                except Exception as e:
                    main_data = pd.DataFrame()
                    print('Error importing dataFrame. Creating new main dataframe')
                    print('\t ERROR : ', e)

                #put the dataframe on a scale 0 to 1 (expect age) and select only non-noise
                datacopy = data.drop('age', axis =1).copy()
                datacopy -= datacopy.min()
                datacopy /= datacopy.max()
                datacopy['age'] = data['age']

               #update dataframe
                main_data = main_data.combine_first(datacopy.fillna(value=0))
                
                #save dataframe
                main_data.to_csv(classMetricsPath, sep = '\t')

    if args['train']:
        if not args['updateDF']:
            try:
                main_data = pd.DataFrame.from_csv(classMetricsPath, sep = '\t', index_col='exp_ic')
                print('Importing dataframe\n------------------------------------')
                print(main_data.head())
            except Exception as e:
                main_data = pd.DataFrame()
                print('Error importing dataFrame')
                print('\t ERROR : ', e)
                assert not main_data.empty, 'Check path to metrics dataframe'

        from sklearn.model_selection import train_test_split
        from sklearn.svm import SVC
        from sklearn.naive_bayes import GaussianNB
        from sklearn.linear_model import LogisticRegression
        from sklearn.ensemble import RandomForestClassifier, VotingClassifier


        new_vars = [#spetial metrics 
                        'spatial.std', 'spatial.min', 'spatial.max', 'mass.perc', 
                        'region.minaxis', 'region.majaxis', 'region.orient', 
                        'threshold.area', 'region.eccentricity',
                        #temporal metrics
                        'temporal.max', 'freq.maxsnr.freq', 'length', 'freq.rangesz']

        y = main_data.loc[:,'signal'].values
        X = main_data.loc[:, main_data.columns != 'signal'].fillna(value=0)

        classConfidence = pd.DataFrame(index=X.index)
        print (data)

        X_train, X_test, y_train, y_test = train_test_split(X[new_vars], y, 
            test_size=0.3, random_state=42)

        #Logistic Regression
        logreg = LogisticRegression(C  = 1, solver = 'liblinear')
        logreg.fit(X_train, y_train)
        y_pred = logreg.predict(X_test)
        print('Accuracy of Logistic Regression classifier on test set:\t\
                {:.2f}'.format(logreg.score(X_test, y_test)))
        classConfidence.loc[X_test.index, 'logreg_prob'] = logreg.predict_proba(X_test)[:,1]
        classConfidence.loc[X_train.index, 'logreg_prob'] = logreg.predict_proba(X_train)[:,1]

        #Gaussian Naive Bayes
        gnb_clf = GaussianNB(var_smoothing= 0.10)
        gnb_clf.fit(X_train, y_train)
        y_pred = gnb_clf.predict(X_test)
        print('Accuracy of Gaussian NB classifier on test set:\t\t\
                {:.2f}'.format(gnb_clf.score(X_test, y_test)))

        classConfidence.loc[X_test.index, 'gnb_prob'] = gnb_clf.predict_proba(X_test)[:,1]
        classConfidence.loc[X_train.index, 'gnb_prob'] = gnb_clf.predict_proba(X_train)[:,1]

        # Support Vector Machine Gaussian Kernal
        svm_clf = SVC(kernel="rbf", gamma=0.01, C=50, random_state=42, probability= True)
        svm_clf.fit(X_train, y_train)
        y_pred = svm_clf.predict(X_test)
        print('Accuracy of SVM classifier on test set:\t\t\t\
                {:.2f}'.format(svm_clf.score(X_test, y_test)))

        classConfidence.loc[X_test.index, 'SVC_prob'] = svm_clf.predict_proba(X_test)[:,1]
        classConfidence.loc[X_train.index, 'SVC_prob'] = svm_clf.predict_proba(X_train)[:,1]

        #Random Forest Classifier
        rnd_clf = RandomForestClassifier(n_estimators = 500, max_features = len(new_vars), random_state=42)
        rnd_clf.fit(X_train, y_train)
        y_pred = rnd_clf.predict(X_test)
        print('Accuracy of Random Forest classifier on test set:\t\
                {:.2f}'.format(rnd_clf.score(X_test, y_test)))
        classConfidence.loc[X_test.index, 'rnd_clf_prob'] = rnd_clf.predict_proba(X_test)[:,1]
        classConfidence.loc[X_train.index, 'rnd_clf_prob'] = rnd_clf.predict_proba(X_train)[:,1]

        #Voting Classifier
        voting_clf = VotingClassifier(
            estimators=[#('lr', logreg), 
                        #('gnb', gnb_clf), 
                        ('rf', rnd_clf), 
                        ('svc', svm_clf)], voting='soft')
        voting_clf.fit(X_train, y_train)
        y_pred = voting_clf.predict(X_test)
        print('Accuracy of Voting classifier on test set:\t\t\
                {:.2f}'.format(voting_clf.score(X_test, y_test)))
        classConfidence.loc[X_test.index, 'voting_clf_prob'] = voting_clf.predict_proba(X_test)[:,1]
        classConfidence.loc[X_train.index, 'voting_clf_prob'] = voting_clf.predict_proba(X_train)[:,1]
        
        g = h5(classifier)
        g.save({'voting_clf':voting_clf, 'training_keys':new_vars })        

        if args['plot']:
            from sklearn.metrics import roc_auc_score
            from sklearn.metrics import roc_curve
            
            classConfidence = 2*(classConfidence - 0.5)
            classConfidence['x'] = np.arange(len(classConfidence))
            start_exp = []
            end_exp = []
            for j, i in enumerate(classConfidence.index.tolist()):
                domain = int(i[-4:])
                if domain == 0:
                    start_exp.append(j)

            for i in start_exp:
                if i != 0:
                    end_exp.append(i-1)
            end_exp.append(len(classConfidence))

            alphas = [0.25, 0.25, 0.25, 0.25, 1] 
            colors = sns.color_palette()
            clfs = ['logreg_prob', 'gnb_prob', 'SVC_prob', 'rnd_clf_prob', 'voting_clf_prob']
            
            classConfidence.loc[X_train.index,'predicted'] = voting_clf.predict(X_train)
            classConfidence.loc[X_test.index,'predicted'] = voting_clf.predict(X_test)
            classConfidence.sort_index()
            classConfidence['marked'] = y
            classConfidence['false'] = classConfidence['predicted'] - classConfidence['marked']

            for i, clf in enumerate(clfs):
                if i == 0:
                    ax1 = classConfidence.plot(kind = 'scatter', x = 'x', y= clf, label = clf,
                                        color = colors[i], alpha = alphas[i], figsize=(20,4), grid=False)
                else:
                    classConfidence.plot(kind = 'scatter', x = 'x', y= clf, label = clf,
                                         color = colors[i], alpha = alphas[i], grid=False, ax = ax1)
            plt.axhline(y = 0, color = 'k', linestyle = '--')
            plt.ylabel('artifact                                  signal')

            xlabel = []
            falsePos = True
            falseNeg = True
            linemax = 0.94

            for j, i in enumerate(classConfidence['false'].tolist()):
                if i < 0:
                    if falseNeg:
                        plt.axvline(x=j, ymax = linemax, color = 'r', alpha=0.5, label = 'False Negative')
                        falseNeg = False
                    else:
                        plt.axvline(x=j, ymax = linemax, color = 'r', alpha=0.5)
                elif i > 0:
                    if falsePos:
                        plt.axvline(x=j, ymax = linemax, color = 'b', alpha=0.5, label = 'False Positive')
                        falsePos = False
                    else:
                        plt.axvline(x=j, ymax = linemax, color = 'b', alpha=0.5)

            for j, i in enumerate(start_exp):
                if (j%2==0):
                    plt.axvspan(xmin=i, xmax =end_exp[j]+1, color='k', alpha=0.1)
                plt.text(i, 1.05, classConfidence.index[i])
            
            # plt.xticks(start_exp, xlabel)
            leg = plt.legend(title = 'Types of Classifier', bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
            leg.get_frame().set_linewidth(0.0)
            plt.xlabel('Num of Domains')
            plt.ylim([-1.05,1.15])           
            plt.show()

            plt.figure()

            logreg.fit(X_train, y_train)

            roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
            plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % roc_auc)

            gnb_clf.fit(X_train, y_train)

            roc_auc = roc_auc_score(y_test, gnb_clf.predict(X_test))
            fpr, tpr, thresholds = roc_curve(y_test, gnb_clf.predict_proba(X_test)[:,1])
            plt.plot(fpr, tpr, label='GaussianNB (area = %0.2f)' % roc_auc)

            svm_clf.fit(X_train, y_train)

            roc_auc = roc_auc_score(y_test, svm_clf.predict(X_test))
            fpr, tpr, thresholds = roc_curve(y_test, svm_clf.predict_proba(X_test)[:,1])
            plt.plot(fpr, tpr, label='SVC (area = %0.2f)' % roc_auc)

            rnd_clf.fit(X_train, y_train)

            roc_auc = roc_auc_score(y_test, rnd_clf.predict(X_test))
            fpr, tpr, thresholds = roc_curve(y_test, rnd_clf.predict_proba(X_test)[:,1])
            plt.plot(fpr, tpr, label='Random Forest Classifier (area = %0.2f)' % roc_auc)

            voting_clf = VotingClassifier(
                estimators=[#('lr', logreg), 
                            #('gnb', gnb_clf), 
                            ('rf', rnd_clf), 
                            ('svc', svm_clf)], voting='soft')
            voting_clf.fit(X_train, y_train)

            roc_auc = roc_auc_score(y_test, voting_clf.predict(X_test))
            fpr, tpr, thresholds = roc_curve(y_test, voting_clf.predict_proba(X_test)[:,1])
            plt.plot(fpr, tpr, label='Voting Classifier (area = %0.2f)' % roc_auc)

            plt.plot([0, 1], [0, 1],'r--')
            plt.xlim([-0.025, 1.0])
            plt.ylim([0, 1.025])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic')
            plt.legend(loc="lower right")
            # plt.savefig('rnd_ROC.svg')
            plt.show()
            
